Задание 3.4

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
boston_data = pd.read_csv('C:/Users/U_M1P8G/Desktop/IDE/Доп. главы высшей математики/Ансамблевые методы/data/boston (1).csv')
boston_data.head()

,crim_rate,zn,business,river,nit_oxiden,rooms,age,dist,highways_index,tax,pup_per_teaс,lower,target
0,"0,00632",18,"2,31",0,"0,538","6,575","65,2","4,09",1,296,"15,3","4,98",24
1,"0,02731",0,"7,07",0,"0,469","6,421","78,9","4,9671",2,242,"17,8","9,14","21,6"
2,"0,02729",0,"7,07",0,"0,469","7,185","61,1","4,9671",2,242,"17,8","4,03","34,7"
3,"0,03237",0,"2,18",0,"0,458","6,998","45,8","6,0622",3,222,"18,7","2,94","33,4"
4,"0,06905",0,"2,18",0,"0,458","7,147","54,2","6,0622",3,222,"18,7","5,33","36,2"


In [7]:
X = boston_data.drop('target', axis=1)  # Признаки (все столбцы кроме целевого)

columns_to_convert = ['crim_rate', 'zn', 'business', 'nit_oxiden', 
                     'rooms', 'age', 'dist', 'pup_per_teaс', 'lower']

X_cleaned = X.copy()

for col in columns_to_convert:
    X_cleaned[col] = X[col].str.replace(',', '.').astype(float)

print(X_cleaned.dtypes)

y = boston_data['target'].str.replace(',', '.').astype(float)  

X_train, X_test, y_train, y_test = train_test_split(
    X_cleaned, y, 
    test_size=0.3,      # 20% данных в тестовую выборку
    random_state=13    
)
print(y_test.mean())

crim_rate         float64
zn                float64
business          float64
river               int64
nit_oxiden        float64
rooms             float64
age               float64
dist              float64
highways_index      int64
tax                 int64
pup_per_teaс      float64
lower             float64
dtype: object
21.973026315789475


In [8]:
X_train.info()


<class 'pandas.core.frame.DataFrame'>
Index: 354 entries, 475 to 338
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   crim_rate       354 non-null    float64
 1   zn              354 non-null    float64
 2   business        354 non-null    float64
 3   river           354 non-null    int64  
 4   nit_oxiden      354 non-null    float64
 5   rooms           354 non-null    float64
 6   age             354 non-null    float64
 7   dist            354 non-null    float64
 8   highways_index  354 non-null    int64  
 9   tax             354 non-null    int64  
 10  pup_per_teaс    354 non-null    float64
 11  lower           354 non-null    float64
dtypes: float64(9), int64(3)
memory usage: 36.0 KB


Обучаем линейную регрессию

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# Создаем и обучаем модель линейной регрессии
lr_model = LinearRegression()  # параметры по умолчанию
lr_model.fit(X_train, y_train)

# Делаем предсказания
y_pred = lr_model.predict(X_test)

# Оцениваем качество модели
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Результаты линейной регрессии:")
print(f"Mean Absolute Error (MSE): {mae:.2f}")
print(f"R² Score: {r2:.4f}")
print(f"Среднее предсказание: {y_pred.mean():.2f}")
print(f"Среднее фактическое: {y_train.mean():.2f}")

Результаты линейной регрессии:
Mean Absolute Error (MSE): 3.72
R² Score: 0.7036
Среднее предсказание: 22.23
Среднее фактическое: 22.77


Обучаем решающее дерево

In [10]:
from sklearn.tree import DecisionTreeRegressor

# Создаем и обучаем решающее дерево
dt_model = DecisionTreeRegressor(random_state=13)  # параметры по умолчанию
dt_model.fit(X_train, y_train)

# Делаем предсказания
y_pred = dt_model.predict(X_test)

# Оцениваем качество модели
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Решающее дерево (параметры по умолчанию):")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.4f}")

Решающее дерево (параметры по умолчанию):
MAE: 2.84
R²: 0.7691


In [11]:
# Делаем предсказания на тренировочной выборке
y_pred_train = dt_model.predict(X_train)

# Оцениваем качество модели
mae = mean_absolute_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print("Решающее дерево (параметры по умолчанию):")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.4f}")

Решающее дерево (параметры по умолчанию):
MAE: 0.00
R²: 1.0000


Обучим 4 случайных леса

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Параметры для моделей
n_trees_list = [3, 10, 100, 500]
best_mae = float('inf')
best_n_trees = 0

print("СЛУЧАЙНЫЙ ЛЕС - СРАВНЕНИЕ")
print("=" * 40)

for n_trees in n_trees_list:
    # Создаем и обучаем модель
    rf_model = RandomForestRegressor(
        n_estimators=n_trees,
        random_state=13
    )
    rf_model.fit(X_train, y_train)
    
    # Предсказания и оценка
    y_pred = rf_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"Деревьев: {n_trees:3d} | MAE: {mae:.4f}")
    
    # Обновляем лучший результат
    if mae < best_mae:
        best_mae = mae
        best_n_trees = n_trees

print("=" * 40)
print(f"Наименьшая MAE: {best_mae:.4f} (при {best_n_trees} деревьях)")

СЛУЧАЙНЫЙ ЛЕС - СРАВНЕНИЕ
Деревьев:   3 | MAE: 2.9329
Деревьев:  10 | MAE: 2.4663
Деревьев: 100 | MAE: 2.2560
Деревьев: 500 | MAE: 2.2366
Наименьшая MAE: 2.2366 (при 500 деревьях)


Случайный лес. Практика

In [13]:
weather_data = pd.read_csv('C:/Users/U_M1P8G/Desktop/IDE/Доп. главы высшей математики/Ансамблевые методы/data/weatherAUS.csv')
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [14]:
weather_data.isna().sum().sum()

np.int64(343248)

In [15]:
# Удаляем признаки с более чем 40% пропусков
threshold = 0.6 * len(weather_data)  # 40% от общего числа строк
weather_cleaned = weather_data.dropna(thresh=threshold, axis=1)

# Считаем сколько признаков было удалено
removed_count = weather_data.shape[1] - weather_cleaned.shape[1]

print(f"Было признаков: {weather_data.shape[1]}")
print(f"Осталось признаков: {weather_cleaned.shape[1]}")
print(f"Удалено признаков: {removed_count}")

Было признаков: 23
Осталось признаков: 20
Удалено признаков: 3


In [16]:
# Создаем словарь для преобразования
mapping = {'Yes': 1, 'No': 0}

# Применяем map с сохранением пропусков
weather_cleaned['RainToday'] = weather_cleaned['RainToday'].map(mapping)
weather_cleaned['RainTomorrow'] = weather_cleaned['RainTomorrow'].map(mapping)

rain_today_mean = weather_cleaned['RainToday'].mean()
print(f"Среднее арифметическое RainToday: {rain_today_mean:.2f}")

Среднее арифметическое RainToday: 0.22


C:\Users\U_M1P8G\AppData\Local\Temp\ipykernel_32756\3581516978.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_cleaned['RainToday'] = weather_cleaned['RainToday'].map(mapping)
C:\Users\U_M1P8G\AppData\Local\Temp\ipykernel_32756\3581516978.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_cleaned['RainTomorrow'] = weather_cleaned['RainTomorrow'].map(mapping)


Обработайте признак Date таким образом, чтобы выделить в отдельный признак Month (номер месяца). Изначальный признак Date удалите. Определите, какой месяц имеет самую большую часть дождливых дней относительно всех дней месяца. В качестве ответа введите порядковый номер месяца.

In [17]:
weather_cleaned['Date'] = pd.to_datetime(weather_cleaned['Date'])
weather_cleaned['month']=weather_cleaned['Date'].dt.month
print("Месяц успешно извлечен!")
print(weather_cleaned[['Date', 'month']].head())
weather_cleaned = weather_cleaned.drop('Date', axis=1)

Месяц успешно извлечен!
        Date  month
0 2008-12-01     12
1 2008-12-02     12
2 2008-12-03     12
3 2008-12-04     12
4 2008-12-05     12


C:\Users\U_M1P8G\AppData\Local\Temp\ipykernel_32756\4115314194.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_cleaned['Date'] = pd.to_datetime(weather_cleaned['Date'])
C:\Users\U_M1P8G\AppData\Local\Temp\ipykernel_32756\4115314194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_cleaned['month']=weather_cleaned['Date'].dt.month


In [18]:
rain_ratio_by_month = weather_cleaned.groupby('month')['RainToday'].mean()
best_month = rain_ratio_by_month.idxmax()

print(f"Месяц с самой большой долей дождливых дней: {best_month}")
print(f"Доля: {rain_ratio_by_month[best_month]:.3f}")

Месяц с самой большой долей дождливых дней: 7
Доля: 0.271


Обработайте оставшиеся категориальные признаки. С помощью метода get_dummies с настройками по умолчанию создайте dummy-переменные для всех категориальных признаков (их пять), которые есть в данных на этот момент.

Кодировку признаков важно выполнить именно в следующем порядке: categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']. Это необходимо для того, чтобы ваши дальнейшие ответы сходились с нашим решением, так как алгоритм случайного леса, который мы будем использовать в дальнейшем, чувствителен к порядку столбцов. Аргумент categoricals передаётся в функцию pd.get_dummies() с использованием ключевого слова columns, а именно columns=categoricals.

In [19]:
categoricals = ['month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']

# Создаем dummy-переменные
weather_encoded = pd.get_dummies(weather_cleaned, columns=categoricals)

# Считаем общее количество признаков (включая целевую переменную)
total_features = weather_encoded.shape[1]

print(f"Количество признаков после кодировки: {total_features}")

Количество признаков после кодировки: 124


In [20]:
weather_final = weather_encoded.dropna()

# Разбиваем на обучающую и тестовую выборки 70/30
X = weather_final.drop('RainTomorrow', axis=1)
y = weather_final['RainTomorrow']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=31
)

# Вычисляем среднее значение целевой переменной на тестовой выборке
test_mean = y_test.mean()

print(f"Среднее значение целевой переменной на тестовой выборке: {test_mean:.2f}")

Среднее значение целевой переменной на тестовой выборке: 0.23


Задание 4.7.

In [47]:
import numpy as np

# Фиксируем случайность
np.random.seed(31)

# Параметры бутстрепа
n_bootstrap = 1000
sample_size = len(X_train)

# Создаем массив для хранения средних значений
bootstrap_means = []

# Генерируем 1000 бутстреп-выборок
for i in range(n_bootstrap):
    # Генерируем случайные индексы с повторениями
    indices = np.random.randint(0, sample_size, sample_size)
    
    # Извлекаем соответствующую выборку MinTemp
    bootstrap_sample = X_train['MinTemp'].iloc[indices]
    
    # Вычисляем среднее и сохраняем
    bootstrap_means.append(bootstrap_sample.mean())

# Вычисляем стандартное отклонение средних
std_dev = np.std(bootstrap_means)

print(f"Стандартное отклонение средних: {std_dev:.2f}")

Стандартное отклонение средних: 0.03


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

clf = LogisticRegression()
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
roc_auc_score(y_test, preds_test)

c:\Users\U_M1P8G\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


np.float64(0.7269592758231629)

In [22]:
# Разбиваем на обучающую и тестовую выборки 70/30
y = weather_final['RainTomorrow']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=31
)

# Вычисляем среднее значение целевой переменной на тестовой выборке
test_mean = y_test.mean()

print(f"Среднее значение целевой переменной на тестовой выборке: {test_mean:.2f}")

Среднее значение целевой переменной на тестовой выборке: 0.23


In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

params = {
    'max_leaf_nodes': list(range(2, 10)), 
    'min_samples_split': [2, 3, 4], 
    'max_depth': [5,7,9,11]
}

grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, y_train)
print(grid_search_cv.best_params_)


Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.817 total time=   0.0s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.825 total time=   0.0s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.817 total time=   0.0s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.825 total time=   0.0s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.817 total time=   0.0s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.820 total time=   0.0s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.825 total time=   0.0s
[CV 1/3] END max_depth=5, max_leaf_nodes=3, min_samples_split=2;,

In [24]:
clf = DecisionTreeClassifier(max_depth = 5, max_leaf_nodes = 9, min_samples_split = 2, random_state=42)
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(y_test, preds_test), 2))

0.7


Задание 4.10

In [29]:
from sklearn.ensemble import RandomForestClassifier

# Создаем и обучаем решающее дерево
dt_model = RandomForestClassifier(random_state=31, n_estimators = 100) 
dt_model.fit(X_train, y_train)

# Делаем предсказания
y_pred = dt_model.predict(X_test)

print(round(roc_auc_score(y_test, y_pred), 2))

0.73


Задание 4.11

In [30]:
params = {'max_features': [ 4, 5, 6, 7]
          , 'min_samples_leaf': [3, 5, 7, 9, 11]
          , 'max_depth': [5, 10, 15]}

grid_search_cv = GridSearchCV(RandomForestClassifier(random_state=31, n_estimators=100), params, verbose=2, cv=3)
grid_search_cv.fit(X_train, y_train)
print(grid_search_cv.best_params_)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=3; total time=   0.9s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=3; total time=   0.8s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=3; total time=   0.8s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=5; total time=   0.8s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=5; total time=   0.9s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=5; total time=   0.8s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=7; total time=   0.7s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=7; total time=   0.8s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=7; total time=   0.8s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=9; total time=   0.7s
[CV] END ....max_depth=5, max_features=4, min_samples_leaf=9; total time=   0.8s
[CV] END ....max_depth=5, max_features=4, min_s

In [31]:
# Создаем и обучаем решающее дерево
dt_model = RandomForestClassifier(random_state=31, n_estimators = 100, max_depth = 15
                                  , max_features = 7, min_samples_leaf = 3) 
dt_model.fit(X_train, y_train)

# Делаем предсказания
y_pred = dt_model.predict(X_test)

print(round(roc_auc_score(y_test, y_pred), 2))

0.7


Задание 4.12. Важность признаков

In [32]:
feature_names = [x for x in weather_final if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': dt_model.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
66,Location_Tuggeranong,0.000000
62,Location_SalmonGums,0.000000
69,Location_Walpole,0.000000
